# "Where did the Village People go?"

## Table of Content
- <a href='#DataMining'>Data Mining</a><br>
- <a href='#DataCleaning'>Data Cleaning</a><br>
- <a href='#DataAnalysis'>Data Analysis</a><br>
- <a href='#Prediction'>Prediction</a><br>
- <a href='#Clustering'>Clustering</a><br>
- <a href='#Conclusion'>Conclusion</a><br>
- <a href='#FutureWorks'>Future Works</a><br>

## Setting Up General Workshop

### General Libraries

In [ ]:
# DataFrames
import pandas as pd
# Basic Plotting for Exploratory Data Analysis
import matplotlib.pyplot as plt
# Fancy Plotting
import seaborn as sns
# Array and Array-Math
import numpy as np
# Geospatial Visualization
import geopandas as gp

### Adding Homebrew Functions
- These Functions are tailored to the Dataset provided by the Bertelsmann Foundation<br>
- The first features of the Dataset are not timeseries. After that, only timeseries of the same length follow.<br>

In [ ]:
# This Function extracts all the features from a given year out of all timesries.
def get_year(year):
    return(FEATURE_TITLES[range(NON_TIMESERIES+year-MIN_YEAR,
                                data_demo_cln.shape[1],
                                MAX_YEAR - MIN_YEAR + 1)])

In [ ]:
# Extract the Time Series of a certain Feature (feature titles)
def get_feature(feature):
    # Where the Features Time Series starts
    start = NON_TIMESERIES+feature*(MAX_YEAR - MIN_YEAR + 1)
    # Return the Columns of Timeseries
    return data_demo_cln.columns[range(start,
                              start+(MAX_YEAR - MIN_YEAR + 1))]

### General Settings

In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]

## Data Mining and Data Cleaning

### Homebrew Functions for Data Mining / Cleaning

In [ ]:
# Filling Missing Data in a Time Series using the existing Data before and after the gap
def fillr3(data):
    # Return Timeseries if gapless
    if(data.isna().sum() == len(data)):
        return(data)    

    # Find the first existing value
    counter = 0
    first_value = 0
    while counter < len(data):
        section = data[:counter]
        if section.isna().sum() == len(section):
            result = counter
        counter +=1
    first_value = result

    # Find the last existing Value
    data = data[::-1]
    counter = 0
    last_value = 0
    while counter < len(data):
        section = data[:counter]
        if section.isna().sum() == len(section):
            result = counter
        counter +=1
    last_value = len(data) - result -1
    data = data[::-1]

    ## Fill the Missing Data with
    # the first existing Datapoint, since the Gap is in beginning
    beg = data.bfill()[:first_value]
    # the mean of the Datapoint before and after the Gap
    mid = ((data.bfill()+data.ffill())/2)[first_value:last_value]
    # the last existing Datapoint, since the Gap is in beginning    
    end = data.ffill()[last_value:]

    return(beg.append(mid.append(end)))

In [ ]:
# This Function Takes the name of a Dataset from Open Data Soft, downloads the GeoJson and turns it into a Geopanda.
def geo_from_ods(title):
    path  = "https://public.opendatasoft.com/explore/dataset/"+title+"/download/?format=geojson"
    geods = gp.read_file(filename = path)
    return geods

<a id='DataMining'></a>
### Data Mining
|Input|Tools & Techniques|Output|
|-|-|-|
|External Data|Import Data|Raw Data|
|||Sources Table|
|||Data Dictionary|

In [ ]:
## Importing Geological Data for different Administrative Levels
geo_nat_raw = geo_from_ods("european-union-countries")
geo_sta_raw = geo_from_ods("bundesland")
geo_dis_raw = geo_from_ods("landkreise-in-germany")

In [ ]:
# Import Demographic Data given by Bertelsmann Stiftung fom Excel File
data_demo_raw = pd.read_excel("bertelsmann.xlsx")

In [ ]:
# Constants derived from properties of Data Set
NON_TIMESERIES = 3
MIN_YEAR       = 2006
MAX_YEAR       = 2017
TIME_SPAN      = MAX_YEAR - MIN_YEAR + 1
FEATURE_TITLES = data_demo_raw.columns
N_TIMESERIES   = 21

In [ ]:
# Handing over Data for Cleaning
data_demo_cln = data_demo_raw.copy()
geo_nat_cln   = geo_nat_raw.copy()
geo_sta_cln   = geo_sta_raw.copy()
geo_dis_cln   = geo_dis_raw.copy()

<a id='DataCleaning'></a>
### Data Cleaning

|Input|Tools & Techniques|Output|
|-|-|-|
|Raw Data|Reworking Features|Clean and Merged Data|
|Data  Dictionary|Data Type Adaption|Data Dictionary Updates|
||Handle Missing Data||
||Merge Datasets||

#### Reworking Features

##### Demographic Data

In [ ]:
## Rename Features
# Choosen Names for Non-Timeseries-Features
id_names   = ["name", "gkz", "demtype"]
# Choosen Names for Timeseries
timeseries = ["pop",
              "nodegree",
              "alevel",
              "immi",
              "emmi",
              "ei_balance",
              "fam_balance",
              "edu_balance",
              "sec_balance",
              "ger_balance",
              "com_in_per_svb",
              "com_out_per_svb",
              "com_blc_per_pop",
              "com_in_per_pop",
              "com_out_per_pop",
              "com_in_svb_fem",
              "com_in_svb_men",
              "com_out_svb_fem",
              "com_out_svb_men",
              "com_blc_pop_fem",
              "com_blc_pop_men"]
# Generate new Feature Names by adding Years
new_columns = id_names
for ts in timeseries:
    for idx in range(MIN_YEAR, MAX_YEAR + 1):
        new_columns.append(ts+"_"+str(idx))
# Rename Features        
data_demo_cln.columns = new_columns

# Adapt Constants
FEATURE_TITLES = data_demo_cln.columns

In [ ]:
## Derive New Features from Existing Ones
# Extract State Number from GKZ
state = np.floor(data_demo_cln.gkz/1000000).astype(int)
# Extract District Number from GKZ
district = np.floor((data_demo_cln.gkz - state*1000000)/1000).astype(int)
# Extract Municipality Number from GKZ
municipality = (data_demo_cln.gkz - state*1000000 - district*1000).astype(int)
# Insert New Information
data_demo_cln.insert(3,"municipality",municipality)
data_demo_cln.insert(3,"district",district)
data_demo_cln.insert(3,"state",state)

# Adapt Constants
NON_TIMESERIES += 3
FEATURE_TITLES = data_demo_cln.columns

In [ ]:
## Add Features ...
# Administrative Level
level = np.full_like(np.arange(data_demo_cln.shape[0], dtype=int), np.nan, dtype=np.double)
level = pd.DataFrame(level)
level.columns = ["level"]
data_demo_cln.insert(0,"level",level)
## .. and fill it
# National Level
deu_index = data_demo_cln[data_demo_cln.name == "Deutschland"].index
data_demo_cln.iloc[deu_index,0]      = 0
# States
sta_index = data_demo_cln[(data_demo_cln.level != 0)&(data_demo_cln.district == 0)].index
data_demo_cln.loc[sta_index,"level"] = 1
# Districts
dis_index = data_demo_cln[(data_demo_cln.level.isna())&(data_demo_cln.municipality == 0)].index
data_demo_cln.loc[dis_index,"level"] = 2
# Municipalities
mun_index = data_demo_cln.level.isna()
data_demo_cln.loc[mun_index,"level"] = 3

# Adapt Constants
NON_TIMESERIES += 1
FEATURE_TITLES = data_demo_cln.columns

##### Geospatial Data

<i> In 2016 the districts of "Göttingen" and "Osterode am Harz" merged into one District named "Göttingen". The Geospatial Data needs to reflect that.</i>

In [ ]:
# Find Göttingen 
indx_goe = geo_dis_cln[geo_dis_cln.name_2 == "Göttingen"].index[0]
# Find Osterode am Harz
indx_oah = geo_dis_cln[geo_dis_cln.name_2 == "Osterode am Harz"].index[0]
# Göttingen and Oseterode am Harz the new District Code
geo_dis_cln.loc[indx_goe,"cca_2"] = '03159'
geo_dis_cln.loc[indx_oah,"cca_2"] = '03159'
# Merge Shapes by Code
geo_dis_cln = geo_dis_cln.dissolve(by='cca_2')
geo_dis_cln = geo_dis_cln.reset_index()

In [ ]:
## Reduce to relevant features, rename and alter them to prepare for merging
# National Level
geo_nat_cln = geo_nat_cln.loc[:,["name_long","geometry"]]
geo_nat_cln.columns = ["name","geo"]
geo_nat_cln.loc[21,"name"] = "Deutschland"
# States Level
geo_sta_cln = geo_sta_cln.loc[:,["gen","geometry"]]
geo_sta_cln.columns = ["name","geo"]
# District Level
geo_dis_cln = geo_dis_cln.loc[:,["cca_2", "geometry"]]
geo_dis_cln.columns = ["code","geo"]

#### Handling Data Type

In [ ]:
## Transform Format
idx_numbers = data_demo_cln.columns[NON_TIMESERIES : data_demo_cln.shape[1]]
for idx in idx_numbers:
    temp_result = []
    for obs in data_demo_cln[idx]:
        if obs == 'k.A.':
            temp_result.append(float('nan'))
        else:
            # Delete delimiter
            result = obs.replace(".","")
            # Turn Comma into Dot
            result = result.replace(",",".")
            # Turn into float
            temp_result.append(float(result))
    data_demo_cln[idx] = temp_result

#### Missing Values

##### Demographic Data - Timeseries

In [ ]:
## Step One: Use existing values to fill missing values
# WARNING - TAKES A MINUTE OR TWO
# Cycle through Timeseries
for idx in range(N_TIMESERIES):
    # Extract Feature-Titles for Timeseries
    focus_on = get_feature(idx)
    # Extract Obervation with at least one value and one missing value
    look_at  = (data_demo_cln[focus_on].isna().sum(axis = 1) > 0)&(data_demo_cln[focus_on].isna().sum(axis = 1) < TIME_SPAN)    
    # Handle Missing Values of selected Timeseries
    data_demo_cln.loc[look_at,focus_on] = data_demo_cln.loc[look_at,focus_on].apply(fillr3, axis = 1)

In [ ]:
## Step Two: Fill Completly empty Cases with mean - for each adminstrative level individually, skip Federal-Level
# Cycle through Administrative Level
for idx_lvl in range(1,4):
    # Number of Observations with selected Administrative Level
    n_obs   = data_demo_cln[data_demo_cln.level == idx_lvl].shape[0]
    # Extract Features with at least one value and one missing value
    fixable = data_demo_cln.columns[(data_demo_cln[data_demo_cln.level == idx_lvl].isna().sum() != 0)&(data_demo_cln[data_demo_cln.level == idx_lvl].isna().sum() != n_obs)]
    # Cycle through those fixabel features    
    for idx_ftr in fixable:
        # Pick Observations from selected Administrative Level
        look_at  = (data_demo_cln.level == idx_lvl)
        # Select Feature
        focus_on = idx_ftr
        # Calculate Filling Value (= mean)
        filler   = data_demo_cln.loc[look_at,focus_on].mean()
        # Fill Missing Values     
        data_demo_cln[idx_ftr].fillna(filler, inplace = True)

#### Merge Data

In [ ]:
# National Level
data_nat = geo_nat_cln.merge(data_demo_cln, on = "name", how = "inner")
data_nat = gp.GeoDataFrame(data_nat, geometry = "geo")

In [ ]:
# State Level
data_sta = geo_sta_cln.merge(data_demo_cln, on = "name", how = "inner")
data_sta = gp.GeoDataFrame(data_sta, geometry = "geo")
# City States
data_cit = data_sta[(data_sta.name == "Hamburg")|(data_sta.name == "Berlin")]
data_cit = gp.GeoDataFrame(data_cit, geometry = "geo")

In [ ]:
# District Level
data_dis = data_demo_cln[data_demo_cln.level==2].copy()
# Generate District Identifying Number, as used in the GeoData
code = np.floor(data_dis.gkz/1000).astype(int).astype(str)
for idx in code.index:
    if len(code[idx]) == 4:
        code[idx] = "0" + code[idx]
    else:
        pass
code         = pd.DataFrame(code)
code.columns = ["code"]
# Add as new Feature
data_dis     = data_dis.join(code, lsuffix='_caller', rsuffix='_other')
# Execute actual merge
data_dis = geo_dis_cln.merge(data_dis, on = "code", how = "inner")
data_dis = gp.GeoDataFrame(data_dis, geometry = "geo")
# Drop temporary Feature
data_dis.drop("code", axis = 1, inplace = True)

In [ ]:
## Visualizing Municipalities geospatially is Future Works
# Municipalities
data_mun = data_demo_cln[data_demo_cln.level == 3]

In [ ]:
## Switch to 3-degree Gauss-Kruger zone 3 Projection
data_nat.crs = {'init':'epsg:4326'}
data_nat     = data_nat.to_crs({'init':'epsg:31467'})
data_sta.crs = {'init':'epsg:4326'}
data_sta     = data_sta.to_crs({'init':'epsg:31467'})
data_cit.crs = {'init':'epsg:4326'}
data_cit     = data_cit.to_crs({'init':'epsg:31467'})
data_dis.crs = {'init':'epsg:4326'}
data_dis     = data_dis.to_crs({'init':'epsg:31467'})

<a id='DataAnalysis'></a>
## Data Analysis

|Input|Tools & Techniques|Output|
|-|-|-|
|Clean and Merged Data|Visualization|Data Understanding|
|||Subject Matter Understanding|

### Single Features

#### Overview of District/ Municipality Sizes

In [ ]:
# Select the Data to show
feat = 0
year = 2017

# Get the Data
show_me = data_dis[get_year(year)[feat]]

# Turn the Data into a Plot
sns.distplot(show_me)
plt.title("Size of Districts")
plt.xlabel("Population")
plt.xticks(rotation=30)
plt.ylabel("Count")
plt.yticks([])

# Save and Show the plot
plt.savefig("./images/SizeOfDistricts.png")
plt.close()
#plt.show()

 <img src="./images/SizeOfDistricts.png"/>

<i> Seen here is the distribution of distict population in 2017. The mean is almost 200k, while districts with more than 700k inhabitants are rare.</i>

In [ ]:
# Select the Data to show
feat = "pop_2017"
year = 2017

# Get the Data
dis_minmax = data_dis.nlargest(5, feat)[["name", feat]]
dis_minmax = dis_minmax.append(data_dis.nsmallest(5, feat)[["name", feat]])
dis_minmax.sort_values(by=[feat], inplace = True, ascending = False)

# Turn the Data into a Plot
sns.barplot(x  = feat, y = "name", data = dis_minmax,  palette="GnBu")
plt.title("The Largest and the Smallest Districts")
plt.xlabel("Population")
plt.xticks(rotation = 30)
plt.xscale("log")
plt.ylabel("")

# Save and Show the plot
plt.tight_layout()
plt.savefig("./images/TopFlopDistricts.png")
plt.close()
#plt.show()

 <img src="./images/TopFlopDistricts.png"/>

<i> Here you see the biggest, and smallest districts in Germany as of 2017. The population is put on a logarithmic scale. The actual values are shown below.</i>

In [ ]:
dis_minmax

In [ ]:
# Select the Data to show
feat = 0
year = 2017

# Get the Data
show_me = data_mun[get_year(year)[feat]]
show_me = show_me[show_me<=100000]

# Turn the Data into a Plot
sns.distplot(show_me)
plt.title("Size of Municipalities")
plt.xlabel("Population")
plt.xticks(rotation=30)
plt.ylabel("Count")
plt.yticks([])


# Save and Show the plot
plt.savefig("./images/SizeOfMunici.png")
plt.close()
#plt.show()

 <img src="./images/SizeOfMunici.png"/>

<i> The Dataset cuts off at 5k inhabitants on the Municipality level. Therefore calculating a mean/ minimum is not representative.</i>

In [ ]:
# Select the Data to show
feat = "pop_2017"
year = 2017

# Get the Data
mun_minmax = data_mun.nlargest(10, feat)[["name",feat]]

# Turn the Data into a Plot
sns.barplot(x  = feat, y = "name", data = mun_minmax,  palette="GnBu")
plt.title("The Largest Municipalites")
plt.xlabel("Population")
plt.xticks(rotation = 30)
plt.ylabel("")

# Save and Show the plot
plt.tight_layout()
plt.savefig("./images/TopMunici.png")
plt.close()
#plt.show()

 <img src="./images/TopMunici.png"/>

<i> However, creating a list of the Top 10 yields useful information</i>

#### Population Development 2006 - 2017

In [ ]:
# Get the Data
show_me = (data_dis.pop_2017 - data_dis.pop_2006)/data_dis.pop_2006*100

# Turn the Data into a Plot
sns.distplot(show_me)
plt.title("District Population Change 2006 - 2017")
plt.xlabel("Change in %")
plt.xticks(rotation=30)
plt.ylabel("Count")
plt.yticks([])

# Save and Show the plot
plt.tight_layout()
plt.savefig("./images/PopChangeDistrict.png")
plt.close()
#plt.show()

 <img src="./images/PopChangeDistrict.png"/>

<i> Seen here the changes in Population</i>

In [ ]:
data_temp = data_dis.copy()
data_temp["changes"] = (data_dis.pop_2017 - data_dis.pop_2006)/data_dis.pop_2006*100

# Get the Data
dis_minmax = data_temp.nlargest(5, 'changes')[["name","changes"]]
dis_minmax = dis_minmax.append(data_temp.nsmallest(5, 'changes')[["name","changes"]])
dis_minmax.sort_values(by=["changes"],inplace = True, ascending = False)

# Turn the Data into a Plot
sns.barplot(x = "changes", y = "name", data = dis_minmax, palette = "GnBu")
plt.title("The Biggest Changes in Population, District")
plt.xlabel("Change in %")
plt.xticks(rotation = 30)
plt.ylabel("")

# Save and Show the plot
plt.tight_layout()
plt.savefig("./images/TopFlopDistrictChanges.png")
plt.close()
#plt.show()

 <img src="./images/TopFlopDistrictChanges.png"/>

<i>Here you can see the highest changes</i>

In [ ]:
## Add City States
data_cit["changes"] = (data_cit.pop_2017 - data_cit.pop_2006)/data_cit.pop_2006*100
data_temp = data_temp.append(data_cit,sort=False).reset_index(drop=True)
data_cit.drop("changes", axis = 1, inplace = True)

# Choose Data to Plot
feat = "changes"

# Center Legend
extrema = data_temp[feat].abs().max()

# Plotting
fig, (ax) = plt.subplots(1, 1)

data_temp.plot(column = feat,
              ax     = ax,
              legend = True,
              cmap   = 'RdYlGn',
              vmax   = extrema,
              vmin   = -extrema)
plt.axis('off')
plt.title("District Population Change 2006 - 2017")
plt.savefig("./images/changemap.png")
plt.close()
#plt.show()

 <img src="./images/changemap.png"/>

<i> The East is loosing its population, but only on in the rural areas. Urban centers nationwide are growing. So are the belts around Germanies Top 3 cities: Berlin, Hamburg and Munich - which are also growing. Cologne's surroundings are not profiting from its growing center.</i>

In [ ]:
# Get the Data
show_me = (data_mun.pop_2017 - data_mun.pop_2006)/data_mun.pop_2006*100

# Turn the Data into a Plot
sns.distplot(show_me)

plt.title("Population Change 2006 - 2017, Municipality")
plt.xlabel("Change in %")
plt.xticks(rotation=30)
plt.ylabel("Count")
plt.yticks([])

# Save and Show the plot
plt.tight_layout()
plt.savefig("./images/PopChangeMunici.png")
plt.close()
#plt.show()

 <img src="./images/PopChangeMunici.png"/>

<i> Seen here are the percentual changes in those 11 years on the municipality level.</i>

In [ ]:
data_temp = data_mun.copy()
data_temp["changes"] = (data_mun.pop_2017 - data_mun.pop_2006)/data_mun.pop_2006*100

# Get the Data
mun_minmax = data_temp.nlargest(5, 'changes')[["name","changes"]]
mun_minmax = mun_minmax.append(data_temp.nsmallest(5, 'changes')[["name","changes"]])
mun_minmax.sort_values(by=["changes"],inplace = True, ascending = False)

# Turn the Data into a Plot
sns.barplot(x = "changes", y = "name", data = mun_minmax, palette = "GnBu")
plt.title("The Biggest Changes in Population, Municipality")
plt.xlabel("Change in %")
plt.xticks(rotation = 30)
plt.ylabel("")

# Save and Show the plot
plt.tight_layout()
plt.savefig("./images/TopFlopMunicipalityChanges.png")
plt.close()
#plt.show()

<img src="./images/TopFlopMunicipalityChanges.png"/>

<i> The biggest winners, and loosers, population wise. Note that Boostedt was one of the smalles municipalities in the first place an barely made the 5k limit.</i>

#### Inner-German Migration by Agegroup

In [ ]:
data_temp = data_dis.copy()
data_temp = data_temp.append(data_cit,sort=False).reset_index(drop=True)

In [ ]:
# Choose Data to Plot
feat = "fam_balance_2017"

# Center Legend
extrema = data_temp[feat].abs().max()

# Plotting
fig, (ax) = plt.subplots(1, 1)

data_temp.plot(column = feat,
              ax     = ax,
              legend = True,
              cmap   = 'RdYlGn',
              vmax   = extrema,
              vmin   = -extrema)
plt.axis('off')
plt.title("Family Migration")
plt.savefig("./images/FamilyMigration.png")
plt.close()
#plt.show()

 <img src="./images/FamilyMigration.png"/>

<i> Families leave the cities for the rural areas.</i>

In [ ]:
# Choose Data to Plot
feat = "edu_balance_2017"

# Center Legend
extrema = data_temp[feat].abs().max()

# Plotting
fig, (ax) = plt.subplots(1, 1)

data_temp.plot(column = feat,
              ax     = ax,
              legend = True,
              cmap   = 'RdYlGn',
              vmax   = extrema,
              vmin   = -extrema)
plt.axis('off')
plt.title("Educational Migration")
plt.savefig("./images/EducationalMigration.png")
plt.close()
#plt.show()

 <img src="./images/EducationalMigration.png"/>

<i> While people 18-24 leave the countryside and flock to the cities.</i>

In [ ]:
# Choose Data to Plot
feat = "sec_balance_2017"

# Center Legend
extrema = data_temp[feat].abs().max()

# Plotting
fig, (ax) = plt.subplots(1, 1)

data_temp.plot(column = feat,
              ax     = ax,
              legend = True,
              cmap   = 'RdYlGn',
              vmax   = extrema,
              vmin   = -extrema)
plt.axis('off')
plt.title("Second Half of Life Migration")
plt.savefig("./images/SECMigration.png")
plt.close()
#plt.show()

 <img src="./images/SECMigration.png"/>

<i> And around 50 the Germans return.</i> 

In [ ]:
# Choose Data to Plot
feat = "ger_balance_2017"

# Center Legend
extrema = data_temp[feat].abs().max()

# Plotting
fig, (ax) = plt.subplots(1, 1)

data_temp.plot(column = feat,
              ax     = ax,
              legend = True,
              cmap   = 'RdYlGn',
              vmax   = extrema,
              vmin   = -extrema)
plt.axis('off')
plt.title("Old Age Migration")
plt.savefig("./images/OldAgeMigration.png")
plt.close()
#plt.show()

 <img src="./images/OldAgeMigration.png"/>

<i> And when reaching the pension-age, people continue to leave the cities - however they also leave certain rural counties for others.</i>

### Two Features
(a quick look into correlations)

In [ ]:
sns.heatmap(data_dis[get_year(2017)].corr().abs())
plt.savefig("./images/CorrDis.png")
plt.close()
#plt.show()

 <img src="./images/CorrDis.png"/>

In [ ]:
sns.heatmap(data_mun[get_year(2017)].corr().abs())
plt.savefig("./images/CorrMun.png")
plt.close()
#plt.show()

 <img src="./images/CorrMun.png"/>

## Predictive Modeling

|Input|Tools & Techniques|Output|
|-|-|-|
|Clean and Merged Data|Visualization|Data Understanding|
||Regression Models|Subject Matter Understanding|
||Clustering||

### Import Necessary Libraries

In [ ]:
# KMeans Clustering
from sklearn.cluster import KMeans
# Evaluate Cluster
from sklearn.metrics import silhouette_score
# KNN Regrssor
from sklearn.neighbors import KNeighborsRegressor
# Model Evaluation via MAE
from sklearn.metrics import mean_absolute_error
# Measuring Time
import time
# Gradient Boosting Regressor
from sklearn.ensemble import GradientBoostingRegressor
# KNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor
# Random Forrest Regressor
from sklearn.ensemble import RandomForestRegressor

### Homebrew Functions

#### Extracting/ Handling the Bertelsmann Dataset

In [ ]:
# Extract certain Feature of a certain year (feature title)
def get_dependent_variable(year):
    return(get_feature(dependent_variable)[(year-MIN_YEAR)])

In [ ]:
# Extract all Features of a certain year-span (feature titles)
def get_years(start_year,end_year):
    # Set up Collector for resulst
    result = []
    #Cycle through years
    while start_year <= end_year:
        # Get Features of selected year, add it to collector
        result.extend(get_year(start_year))  
        # Prepare next iteration
        start_year += 1
    # Return Collector
    return(result)

In [ ]:
# Extract independent Features' Data for certain Timespan
def indep_data_from_years(start_year,end_year):
    # Extract Features' Titles
    independent_features_pred = get_years(start_year,end_year)
    # Get (in)dependent Data
    result = data[independent_features_pred]
    return(result)

In [ ]:
# Extract dependent Features' Data for certain point of time
def dep_data_from_years(year):
    # Get independent Feature Title            
    dependent_feature = get_dependent_variable(year)        
    # Get (in)dependent Data
    result = data[dependent_feature]
    return(result)

#### Working with my Prediction approach: I - Setting the Foundations

In [ ]:
## Generate a DataFrame with the possible Combinations of "Xt + Xtuse-1 = Yt+use-1+lif"
# for given "use"/"lif", used when creating the model
def generate_ulc_year_combos_model(use, lif):
 
    # Set up Collector for Results
    result = []
    # Starting Year of Data
    counter = MIN_YEAR
    # Cycle through possible Combinations
    while counter + use + lif <= MAX_YEAR + 1:
        # Add Results (starting year, ending year, prediction year) to Collector
        result.append([(counter),(counter + use - 1),(counter + use + lif -1 )])
        # Set up for potential next Iteration of Loop
        counter += 1  
    # Turn Results into DataFrame
    result = pd.DataFrame(result)
    # Name Collumns
    result.columns = ["start_year","end_year","predict_year"]
    # Return Dataframes with possible Combinations
    
    return(result)

In [ ]:
## Generates all the possible "use" / "look into future" combinatios, given the timespan of 
# the available data
def generate_use_lif_combinations():

    # Collector for Results
    result = []
    # The Minimum Ammount of Years to utilize as dependent data
    use = 1
    # The Minimum Ammount of Years to predict into the future to
    lif = 1
    # Cycle through all the combinations of "use"
    while use + lif <= TIME_SPAN:
        # Cycle through all the combinations of "lif"               
        while use + lif <= TIME_SPAN:
            # Store resulting combination in collector
            result.append([use,lif])
            # Prepare next iteration
            lif += 1
        # Prepare nex Iteration
        lif  = 1
        use += 1

    # Turn result into DataFrame
    result = pd.DataFrame(result)
    # Name Columns
    result.columns = ["use","lif"]
    # Return Result
    return(result)

#### Working with my Prediction approach: II - Building the Market

In [ ]:
# Make Prediction with given model and timespan
def make_pred(model, start_year, end_year):
    # Get (in)dependent Data from Years for Predictions
    X_pred = indep_data_from_years(start_year,end_year)   
    # Make Predictions
    y_pred = model.predict(X_pred)
    # Return Values
    return(y_pred)

In [ ]:
# Set up the Collection of Models to choose the best from later
def set_up_market(regressor, parameters, show_me):
    start = time.time()
    
    # Loop Level 1: All Combinations of USE / LIF
    ulc = generate_use_lif_combinations()   
    # Set up Collectors for Results
    results  = []       
    for idx in ulc.index:
        use = ulc.use[idx]
        lif = ulc.lif[idx]
        ulc_yc = generate_ulc_year_combos_model(use, lif)

        
        # If show_me is True, the Rhombus-Diagram of each LIF / USE Combination will be set up,
        # to be filled and plotted later
        if show_me:
            canvas_small   = pd.DataFrame(np.full(((MAX_YEAR - MIN_YEAR),
                                                   (2 * (MAX_YEAR - MIN_YEAR) - 1)),
                                                  np.nan))
            canvas_small.index   = range(MIN_YEAR + 1 ,MAX_YEAR + 1)
            canvas_small.columns   = range(-10,11)
 

        # Loop Level 2: All Combinations of Years (independent and dependent Data) to fit model
        for idxidx in ulc_yc.index:
            start_year_model   = ulc_yc.start_year[idxidx]
            end_year_model     = ulc_yc.end_year[idxidx]
            predict_year_model = ulc_yc.predict_year[idxidx]
            # Get (in)dependent Feature Titles
            independent_features_model  = get_years(start_year_model,
                                                    end_year_model)
            dependent_feature_model     = get_dependent_variable(predict_year_model)
            # Get (in)dependent Data
            X_model = data[independent_features_model]
            y_model = data[dependent_feature_model]
     
            # Choose Model to set up
            model = regressor(**parameters)       
            model.fit(X_model, y_model)             

                
            # Loop Level 3: Try Model on all known Data for Scoring
            for idxidxidx in ulc_yc.index:
                # Extract Years for Prediction
                start_year_pred   = ulc_yc.start_year[idxidxidx]
                end_year_pred     = ulc_yc.end_year[idxidxidx]
                predict_year_pred = ulc_yc.predict_year[idxidxidx]        
                # Determine "Push": How many Years forward from fitting was the model used
                push = predict_year_pred - predict_year_model           
                
                # Get y True
                y_true = dep_data_from_years(predict_year_pred)
                # Make Prediction with given model and timespan
                y_pred = make_pred(model, start_year_pred, end_year_pred)

                # Determine score
                score = scoring(y_true, y_pred)
            
                # Collector for Results
                results.append([use,lif,
                                start_year_model, end_year_model, predict_year_model,
                                push,
                                start_year_pred, end_year_pred, predict_year_pred,
                                score,
                                model])                           
      
                # If show_me is True, the set-up Rhombus-Diagram of each LIF / USE Combination will be filled,
                # to be plotted later
                if show_me:
                    canvas_small.loc[predict_year_model,push] = score


        # If show_me is True, the set up and filled Rhombus-Diagram of each LIF / USE Combination is plotted  
        if show_me:
            plt.figure(figsize=(10,10))
            plt.title("Using "+str(use)+" years to predict "+str(lif)+" years in the future")
            sns.heatmap(canvas_small, annot = False, linewidth=0.0, cbar = True, cmap="YlGnBu")
            plt.xlabel("Model pushed ahead")
            plt.ylabel("Original Prediction Year")       
            plt.show()

    # Turn Results Collector into Panda           
    results          = pd.DataFrame(results)
    results.columns  = ["use","lif",
                        "start_year_model", "end_year_model", "predict_year_model","push",
                        "start_year_pred", "end_year_pred", "predict_year_pred",
                       "score", "model"]
    
    # Take Time
    passed = time.time() - start
    
    # Return Results
    return(passed, results)

In [ ]:
# Processes the Market
def prune_market(rfe, shape):
    # The Shape determinses which tests are used to determine the value of the model
    if shape == "triangle":
        rfe = rfe[rfe.push>0]
    else:
        pass
    return rfe

In [ ]:
def best_selection(rfe,shape,show_me,cut_off,name):
    start = time.time()

    # Select the relevant tests to determine the value of each model
    rfe = prune_market(rfe, shape)

    # Calculate the Mean Score for each model
    rfe_mean = rfe.groupby(["use","lif","start_year_model","end_year_model","predict_year_model"], as_index = False)["score"].mean()
 
    # Best score for each lif
    rfe_sum_min = rfe_mean.groupby(["lif"], as_index = False)["score"].min()

    # Calculate the mean Score
    mean_score = rfe_sum_min.score.mean()
    
    # If show_me = True, the Scores of the best Selection will be Plotted
    if show_me:
        plt.plot(rfe_sum_min.score)
        plt.plot(np.full((12, 1), mean_score))        
        plt.ylabel("scoring")
        plt.xlabel("Years predicted ahead")
        plt.xticks(range(0,11),range(1,12))
        # To enable multiple Scores in  one Plot
        if name != "":
            plt.savefig("./images/"+name)
            plt.close()    
        if cut_off:
            plt.show()
    
    # Calculate the mean Score
    
    # Create List of Indexes, leading to the best scoring model for each lif
    # Future Works: Make this more elegant
    best_combo = []
    for idx in rfe_sum_min.index:
        #rfe_mean[rfe_sum_min.score[idx]==rfe_mean.score]
        sym = int(rfe_mean[rfe_sum_min.score[idx]==rfe_mean.score].start_year_model.reset_index(drop=True)[0])
        eym = int(rfe_mean[rfe_sum_min.score[idx]==rfe_mean.score].end_year_model.reset_index(drop=True)[0])
        pym = int(rfe_mean[rfe_sum_min.score[idx]==rfe_mean.score].predict_year_model.reset_index(drop=True)[0])
        best_id = rfe[(rfe.start_year_model   == sym)&
        (rfe.end_year_model     == eym)&
        (rfe.predict_year_model == pym)].index[0]
        best_combo.append(best_id)
    
    # Return Results
    passed = time.time() - start    
    return(passed,best_combo,mean_score)

### Setting up and Selecting Prediction Model

In [ ]:
## Settings, chosen by User
# Used DataSet
data               = data_dis
# Dependent Variable
dependent_variable = 0
# Scoring Method
scoring            = mean_absolute_error

#### Quick Look into KNR

In [ ]:
# Regression Model
regressor          = KNeighborsRegressor
# Model Parameters
parameters         = {"n_neighbors" : 2, "algorithm" : "brute"}
# Set up Model Market
time_passed_one, model_market = set_up_market(regressor, parameters, False)
# Choose the Best Combination
time_passed_two, best_combo, mean_score = best_selection(model_market,"triangle",True,True,"KNregressor.png")
#print("Time passed:", time_passed_one+time_passed_two)

<img src="./images/knregressor.png"/>

<i>KNN is fast (15s) and gives decent results. Graph shows performance for different lif. And mean.</i>

#### Applying approach to Random Forrest Regressor

In [ ]:
time_to_build = 0
counter = 2
rfr_results = []
while time_to_build < (2 * 60):
    # Regression Model
    regressor          = RandomForestRegressor
    parameters         = {"n_estimators" : counter}
    # Set up Model Market
    time_passed_one, model_market = set_up_market(regressor, parameters, False)
    # Choose the Best Combination
    time_passed_two, best_combo, mean_score = best_selection(model_market,"triangle",False,False)
    time_to_build = time_passed_one+time_passed_two
    rfr_results.append([counter,time_to_build,mean_score])        
    counter += 1
rfr_results         = pd.DataFrame(rfr_results)
rfr_results.columns = ["n_estimators","time","mean_score"]

In [ ]:
# Plot Results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
fig.suptitle('Random Forest Regressor')

# Plot Score per Number of Estimator
ax1.plot(rfr_results.n_estimators,rfr_results.mean_score)
ax1.set_ylabel("Average Score")
ax1.set_xlabel("Number of", rotation = 45)

# Plot Score per Time to fit Model
ax2.scatter(x = rfr_results.time, y = rfr_results.mean_score, s = rfr_results.n_estimators)
ax2.set_xticklabels("")
ax2.set_yticklabels("")

ax3.axis('off')

# Plot Number of Estimators and the Time it takes to fit 
ax4.plot(rfr_results.time,rfr_results.n_estimators)
ax4.set_xlabel("Time in Seconds")
ax4.set_ylabel("Estimators", rotation = 45)

# Handle Plot
plt.figure(figsize=(50,50))
fig.savefig("images/RFR.png")
plt.close()
#plt.show()

<img src="./images/RFR.png"/>

<i>The Random Forest Regressor yields better results, but takes more time. The more trees involved, the better the performance / the longer it takes to set the model up. However, the results are still "wobbly" and the performance quickly plateaus.</i>

#### Applying Approach to Gradient Boost Regressor

In [ ]:
time_to_build = 0
counter = 2
gbr_results = []
while time_to_build < (2 * 60):
    # Regression Model
    regressor          = GradientBoostingRegressor
    parameters         = {"n_estimators" : counter}
    # Set up Model Market
    time_passed_one, model_market = set_up_market(regressor, parameters, False)
    # Choose the Best Combination
    time_passed_two, best_combo, mean_score = best_selection(model_market,"triangle",False,False)
    time_to_build = time_passed_one+time_passed_two
    gbr_results.append([counter,time_to_build,mean_score])        
    counter += 1
gbr_results         = pd.DataFrame(gbr_results)
gbr_results.columns = ["n_estimators","time","mean_score"]

In [ ]:
# Plot Results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
fig.suptitle('Gradient Boosting Regressor')

# Plot Score per Number of Estimator
ax1.plot(gbr_results.n_estimators,gbr_results.mean_score, color = "orange")
ax1.set_ylabel("Average Score")
ax1.set_xlabel("Number of", rotation = 45)

# Plot Score per Time to fit Model
ax2.scatter(x = gbr_results.time, y = gbr_results.mean_score, s = gbr_results.n_estimators, color = "orange")
ax2.set_xticklabels("")
ax2.set_yticklabels("")

ax3.axis('off')

# Plot Number of Estimators and the Time it takes to fit 
ax4.plot(gbr_results.time,gbr_results.n_estimators, color = "orange")
ax4.set_xlabel("Time in Seconds")
ax4.set_ylabel("Estimators", rotation = 45)

# Handle Plot
plt.figure(figsize=(50,50))
fig.savefig("images/GBR.png")
plt.close()
#plt.show()

 <img src="images/GBR.png"/>

<i> The Gradient Boost Regressor starts with a very bad score, but improves quickly.<br>
To decide, a closer look needs to be made.</i>

In [ ]:
# Plot GBR
sns.scatterplot(data = gbr_results[gbr_results.time>=60], x = "time", y = "mean_score",
                color = "orange", size = "n_estimators", legend = False)
# Plot RFR
sns.scatterplot(data = rfr_results[rfr_results.time>=60], x = "time", y = "mean_score",
                color = "blue", size = "n_estimators", legend = False)

# Set Frame
plt.title("Comparing Random Forrest with Gradient Boost")
plt.ylabel("Average Score")
plt.xlabel("Time in Seconds")
plt.savefig("images/RFRvsGBR.png")
plt.close()
#plt.show()

 <img src="images/RFRvsGBR.png"/>

<i> Around 80 seconds is the turning point. If more time is invested in setting up the model, Gradient Boost will outperform the Random Forrest. GBR also has not plateaued yet!</i>

### Set up Selected Model

<i> After this step, there will be a collection of models to predict the Migration Balance and the Population on the Municipality and the District Level. For each there is an optimal model for each "lif" that needs to be chosen by the user.</i>

In [ ]:
# There is no educational Data on the Municipalities. However, GDB does not handle missing values.
# To keep things modular, the features will simply be turned tu "0".
data_mun.loc[:,get_feature(1)] = 0
data_mun.loc[:,get_feature(2)] = 0

In [ ]:
## Settings, chosen by User
# Used DataSet
data               = data_mun
# Dependent Variable
dependent_variable = 0
# Scoring Method
scoring            = mean_absolute_error
# Regression Model
regressor          = GradientBoostingRegressor
parameter          = {"n_estimators" : 200}
# Set up Model Market
time_passed_one, mm_mun_pop = set_up_market(regressor, parameter, False)
# Choose the Best Combination
time_passed_two, bc_mun_pop, ms_mun_pop = best_selection(mm_mun_pop,"triangle",True,True,"MUNpop.png")
print("Time passed:", time_passed_one+time_passed_two)

<img src="./images/MUNpop.png"/>

<i>It took roughly 38 minutes to set up this model-series for predicting the migration balance on a municipality level. 200 estimators were used.</i>

In [ ]:
## Settings, chosen by User
# Used DataSet
data               = data_mun
# Dependent Variable
dependent_variable = 5
# Scoring Method
scoring            = mean_absolute_error
# Regression Model
regressor          = GradientBoostingRegressor
parameter          = {"n_estimators" : 200}
# Set up Model Market
time_passed_one, mm_mun_mibal = set_up_market(regressor, parameter, False)
# Choose the Best Combination
time_passed_two, bc_mun_mibal, ms_mun_mibal = best_selection(mm_mun_mibal,"triangle",True,True,"MUNmibal.png")
print("Time passed:", time_passed_one+time_passed_two)

<img src="./images/MUNmibal.png"/>

<i>It took roughly 38 minutes to set up this model-series for predicting the population on a municipality level. 200 estimators were used.</i>

In [ ]:
## Settings, chosen by User
# Used DataSet
data               = data_dis
# Dependent Variable
dependent_variable = 0
# Scoring Method
scoring            = mean_absolute_error
# Regression Model
regressor          = GradientBoostingRegressor
parameter          = {"n_estimators" : 2000}
# Set up Model Market
time_passed_one, mm_dis_pop = set_up_market(regressor,parameter, False)
# Choose the Best Combination
time_passed_two, bc_dis_pop, ms_dis_pop = best_selection(mm_dis_pop, "triangle",True,True,"DISpop.png")
print("Time passed:", time_passed_one+time_passed_two)

<img src="./images/DISpop.png"/>

<i>It took roughly an hour to set up this model-series for predicting the population on a municipality level. 2000 estimators were used.</i>

In [ ]:
## Settings, chosen by User
# Used DataSet
data               = data_dis
# Dependent Variable
dependent_variable = 5
# Scoring Method
scoring            = mean_absolute_error
# Regression Model
regressor          = GradientBoostingRegressor
parameter          = {"n_estimators" : 1000}
# Set up Model Market
time_passed_one, mm_dis_mibal = set_up_market(regressor,parameter, False)
# Choose the Best Combination
time_passed_two, bc_dis_mibal, ms_dis_mibal = best_selection(mm_dis_mibal, "triangle",True,True,"DISmibal.png")
print("Time passed:", time_passed_one+time_passed_two)

<img src="./images/DISmibal.png"/>

<i>It took roughly 29 minutes to set up this model-series for predicting the migration balance on a municipality level. 1000 estimators were used.</i>

### Make Predictions

In [ ]:
## Generate a DataFrame with the possible Combinations of "Xt + Xtuse-1 = Yt+use-1+lif"
# for given "use"/"lif", used when making predictions
def generate_ulc_year_combos_pred(use, lif):
    # Set up Collector for Results
    result = []
    # Starting Year of Data
    counter = MIN_YEAR + use - 1
    # Cycle through possible Combinations
 
    while counter  <= MAX_YEAR:
        # Add Results (starting year, ending year, prediction year) to Collector
        result.append([(counter - use + 1),(counter),(counter + lif)])
        # Set up for potential next Iteration of Loop
        counter += 1  

    # Turn Results into DataFrame
    result = pd.DataFrame(result)
    # Name Columns
    result.columns = ["start_year","end_year","predict_year"]
    # Return Dataframes with possible Combinations 
    return(result)

In [ ]:
# Make Pedictions, with given mode (via best_combo/ model_market) and chosen range (lif)
def predictionS(lif,best_combo,model_market):
    # Extract Tools
    model_idx = best_combo[lif-1]
    use       = model_market.loc[model_idx,"use"]
    model     = model_market.loc[model_idx,"model"]
    
    # Generate Combinations for Predictions
    ulcycp    = generate_ulc_year_combos_pred(use, lif)
    
    # Set up Collector for Predictions
    results = []
    results = pd.DataFrame(results)
    # Make Predictions
    for idx in ulcycp.index:
        # Extract Info for Predictions
        start_year     = ulcycp.loc[idx,"start_year"]
        end_year       = ulcycp.loc[idx,"end_year"]
        predict_year   = ulcycp.loc[idx,"predict_year"]        
        # Make Predictions
        y_pred         = pd.DataFrame(make_pred(model, start_year, end_year))
        results[[str(predict_year)]] = y_pred
        
    results.columns = range(int(results.columns[0]), int(results.columns[-1]) +1)
    return(results)

In [ ]:
# Shows one Example Prediction
def show_example(data,feature,lif,best_combo,model_market,look_at,overlapse,filename,noshow):
    plt.title(look_at+", "+timeseries_n[feature])
    pred = predictionS(lif,best_combo,model_market)
    look_at = data[data.name == look_at].index[0]
    data.reset_index(inplace = True, drop = True)
    past = data.loc[look_at,get_feature(feature)]
    past.index = range(2006,2018)
    if overlapse:
        future = pred.loc[look_at,:]
        plt.plot(past, color = "blue")
        plt.plot(future, color = "orange")     
    else:
        future = pred.loc[look_at,2018:]  
        future = past.append(future)
        plt.plot(future, color = "orange")
        plt.plot(past, color = "blue")
    plt.xticks(range(2006,2017+lif+1),rotation=45)
    plt.savefig(filename)
    if noshow:
        plt.close()
    else:
        plt.show()

In [ ]:
timeseries_n = ["Population",
              "% no degree",
              "% with a-Level",
              "immigration per 1000",
              "emmigration per 1000",
              "migration balance per 1000"]

#### Municipalities

<i> Trials and visual exploration showed that looking into the future (lif) for certain numbers of years yields no plausible results. Here are graphs that show some more prominent Municipalities/ Districts.</i>

In [ ]:
show_example(data_mun,5,10,bc_mun_mibal, mm_mun_mibal,"Hannover",False,"./images/HanMigbal7.png",True)

 <img src="images/HanMigbal7.png"/>

<i> Hannovers stream of Migration will become steady.</i>

In [ ]:
show_example(data_mun,5,10,bc_mun_mibal, mm_mun_mibal,"Plön (PLÖ)", False,"./images/PLOMigbal7.png",True)

 <img src="images/PLOMigbal7.png"/>

In [ ]:
show_example(data_mun,5,10,bc_mun_mibal, mm_mun_mibal,"Unterföhring", False,"./images/UNTMigbal7.png",True)

 <img src="images/PLOMigbal7.png"/>

In [ ]:
show_example(data_mun,5,6,bc_mun_mibal, mm_mun_mibal,"Eisenhüttenstadt", False,"./images/EISMigbal7.png",True)

 <img src="images/EISMigbal7.png"/>

<i> Eissenhüttenstadt will recover.</i>

In [ ]:
show_example(data_mun,0,6,bc_mun_pop, mm_dis_pop,"Aachen", False, "./images/AacPop7.png",True)

 <img src="images/AacPop7.png"/>

<i> The Municipality of Aachen will regain it's population that it lost during the 2011 census in one year.</i><br>
<i> Hm. </i>

#### Population in Districs

In [ ]:
show_example(data_dis,5,7,bc_dis_mibal, mm_dis_mibal,"Suhl", False, "./images/SUHmibal7.png",True)

 <img src="images/SUHpop7.png"/>

<i> The migration balance of Suhl will plateau around 1,5%.</i>

In [ ]:
show_example(data_dis,5,7,bc_dis_mibal, mm_dis_mibal,"Zweibrücken", False, "./images/ZWEmibal7.png",True)

 <img src="images/ZWEpop7.png"/>

<i>So will Zweibrückens.</i>

In [ ]:
show_example(data_dis,5,7,bc_dis_mibal, mm_dis_mibal,"Region Hannover", False, "./images/HANmibal7.png",True)

 <img src="images/HANpop7.png"/>

<i> The District "Region of Hannover" will continue to attract more and more people.</i>

In [ ]:
show_example(data_dis,5,7,bc_dis_mibal, mm_dis_mibal,"Potsdam", False, "./images/POTmibal7.png",True)

 <img src="images/POTpop7.png"/>

<i> The District of Potsdam will, within one year, drastically loose it's attracting forces. However, it will slowly recuperate.</i><br>
<i>Hm.</i>

In [ ]:
show_example(data_dis,0,10,bc_dis_pop, mm_dis_pop,"Potsdam", False, "./images/POTpop7.png",True)

 <img src="./images/POTpop7.png"/>

<i> The fastest growing district 06-17 will continue to grow.</i>

In [ ]:
show_example(data_dis,0,10,bc_dis_pop, mm_dis_pop,"München", False, "./images/MUNpop7.png",True)

 <img src="./images/MUNpop7.png"/>

<i>However, Munich, the biggest District, population will have peaked in 2016 and continue its downfall.</i><br>
<i>Hm.</i>

In [ ]:
show_example(data_dis,0,10,bc_dis_pop, mm_dis_pop,"Region Hannover", False, "./images/HANpop7.png",True)

 <img src="./images/HANpop7.png"/>

<i> The Region of Hannover - Germany's second biggest District - will remain steady until 2024 and then grow again.</i> 

In [ ]:
show_example(data_dis,0,10,bc_dis_pop, mm_dis_pop,"Suhl", False, "./images/SUHpop7.png",True)

<img src="./images/SUHpop7.png"/>

<i> Suhl, the second smallest district in Germany will continue to decline until 2023, but will recover within little time.</i><br>
<i>Hm.</i>

In [ ]:
show_example(data_dis,0,10,bc_dis_pop, mm_dis_pop,"Zweibrücken", False, "./images/ZWEpop7.png",True)

 <img src="./images/ZWEpop7.png"/>

<i>Zweibrücken, the smallest District, will have an interesting development. After stagnating, the population will jumpstart by gaining more then 10% in one year and continue to grow afterwards.</i><br>
<i>Hm.</i>

In [ ]:
show_example(data_dis,0,10,bc_dis_pop, mm_dis_pop,"Münster", False, "./images/MUEpop7.png",True)

 <img src="./images/MUEpop7.png"/>

<i>Münster, Germany's second fastes growing district, will continue to do so after a short slump.</i>

In [ ]:
show_example(data_dis,0,10,bc_dis_pop, mm_dis_pop,"Spree-Neiße, LK", False, "./images/SNKpop7.png",True)

 <img src="./images/SNKpop7.png"/>

<i>Germany's second fastest shrinking district will continue to do so.</i> 

In [ ]:
show_example(data_dis,0,10,bc_dis_pop, mm_dis_pop,"Suhl", False, "./images/SHLpop7.png",True)

 <img src="./images/SHLpop7.png"/>

<i>Suhl, the fastest shrinking district however will see an population explosion in 2023.</i><br>
<i>Hm.</i>

In [ ]:
lif = 10
plt.title("Germany's Population, excluding Berlin and Hamburg")
past = data_dis[get_feature(0)].sum()
past.index = range(2006,2018)
future = predictionS(10,bc_dis_pop,mm_dis_pop).sum()
future = future.loc[2018:]  
future = past.append(future)
plt.plot(future, color = "orange")
plt.plot(past, color = "blue")
plt.xticks(range(2006,2017+lif+1),rotation=45)
plt.savefig("./images/generalpop.png")
plt.close()
#plt.show()

<img src="./images/generalpop.png"/>

<i>Summing up all districts shows that the population in Germany (excluding Hamburg and Berlin) will rise, after slumping till 2021.</i><br>
<i>Hm.</i>

In [ ]:
now  = data_dis[get_feature(0)]["pop_2017"]
soon = predictionS(10,bc_dis_pop,mm_dis_pop).iloc[:,-1]

data_temp = data_dis.copy()
data_temp["changes"] = (soon - now) / now * 100

# Get the Data
dis_minmax = data_temp.nlargest(5, 'changes')[["name","changes"]]
dis_minmax = dis_minmax.append(data_temp.nsmallest(5, 'changes')[["name","changes"]])
dis_minmax.sort_values(by=["changes"],inplace = True, ascending = False)

# Turn the Data into a Plot
sns.barplot(x = "changes", y = "name", data = dis_minmax, palette = "GnBu")
plt.title("The Biggest Changes in Population 2017-2027, District")
plt.xlabel("Change in %")
plt.xticks(rotation = 30)
plt.ylabel("")

# Save and Show the plot
plt.tight_layout()
plt.savefig("./images/TopFlopDistrictPred.png")
plt.close()
#plt.show()

<img src="./images/TopFlopDistrictPred.png"/>

<i> Here you can see the fastest growing/ shrinking districts.</i>

In [ ]:
## Add City States
data_cit["changes"] = 0
data_temp = data_temp.append(data_cit,sort=False).reset_index(drop=True)
data_cit.drop("changes", axis = 1, inplace = True)

# Choose Data to Plot
feat = "changes"

# Center Legend
extrema = data_temp[feat].abs().max()

# Plotting
fig, (ax) = plt.subplots(1, 1)

data_temp.plot(column = feat,
              ax     = ax,
              legend = True,
              cmap   = 'RdYlGn',
              vmax   = extrema,
              vmin   = -extrema)
plt.axis('off')
plt.title("District Population Change 2017 - 2027")
plt.savefig("./images/changemappred.png")
plt.close()
#plt.show()

<img src="./images/changemappred.png"/>

<i> As it can be seen here, the east will either stabilize, or even grow - besides some exceptions.</i><br>
<i>Hm.</i>

In [ ]:
now  = data_mun[get_feature(0)]["pop_2017"]
soon = predictionS(10,bc_mun_pop,mm_mun_pop).iloc[:,-1]

data_temp = data_mun.copy()
data_temp["changes"] = (soon - now) / now * 100

# Get the Data
mun_minmax = data_temp.nlargest(5, 'changes')[["name","changes"]]
mun_minmax = mun_minmax.append(data_temp.nsmallest(5, 'changes')[["name","changes"]])
mun_minmax.sort_values(by=["changes"],inplace = True, ascending = False)

# Turn the Data into a Plot
sns.barplot(x = "changes", y = "name", data = mun_minmax, palette = "GnBu")
plt.title("The Biggest Changes in Population 2017-2027, Municipality")
plt.xlabel("Change in %")
plt.xticks(rotation = 30)
plt.ylabel("")

# Save and Show the plot
plt.tight_layout()
plt.savefig("./images/TopFlopMunicipalityPred.png")
plt.close()
#plt.show()

<img src="./images/TopFlopMunicipalityPred.png"/>

<i> Here you can see the fastest growing/ shrinking municipalities.</i>

### Clustering

<i> Let's see what if there are any patterns in the age-structure of the inner-german migration.</i>

#### Determine Number of Clusters

In [ ]:
# Choose Data to Cluster
cluster_me = data_dis[["fam_balance_2017","edu_balance_2017","sec_balance_2017","ger_balance_2017"]]

# Set Up Collector for Results
results = []
# Cycle through different number of clusters
for idx in range(2,12):

    # Cluster Data by applyinge KMeans
    kmeans = KMeans(n_clusters = idx)
    kmeans.fit(cluster_me)
    y_kmeans = kmeans.predict(cluster_me)
    
    # Sort Data
    counter = 0
    micro_vault = []
    while counter < idx:
        result = pd.DataFrame(cluster_me[y_kmeans == counter])
        micro_vault.append(result)
        counter +=1
    
    # Calculate Silouette Score, and save result in Collector
    results.append(silhouette_score(X = cluster_me, labels = y_kmeans))

In [ ]:
# Set-Up Multi-Plot
f, (leftplot, rightplot) = plt.subplots(1, 2)

# Plot of Silhouette Score
leftplot.set_xlabel("Number of Clusters")
leftplot.set_ylabel("Silhouette Score")
leftplot.set_title("Comparing Silhouette Score")
leftplot.set_xticks(range(0,10))
leftplot.set_xticklabels(range(2,12))
leftplot.plot(results);

# Plot of Silhouette Score Gains
rightplot.set_title("Gains by additional Cluster")
rightplot.set_xlabel("Number of Clusters")
rightplot.set_xticks(range(0,10))
rightplot.set_xticklabels(range(3,11))
rightplot.plot(np.diff(results));

# Handle Results
plt.tight_layout()
plt.savefig("./images/ScoringClusters.png")
plt.close()
#plt.show()

<img src="./images/ScoringClusters.png" />

<i>As it can be seen here, the gains collapse after 4 clusters. Therefore I choose to cluster the Distircts into 4 clusters.</i>

#### Cluster

In [ ]:
n = 4

In [ ]:
# Cluster Data
kmeans = KMeans(n_clusters = n)
kmeans.fit(cluster_me)
y_kmeans = pd.Series(kmeans.predict(cluster_me))

In [ ]:
# Set-Up Multi-Plot
f, (leftplot, rightplot) = plt.subplots(1, 2)

leftplot.set_title("Districts Migration Age Profiles")
counter = 0
while counter < n:
    leftplot.plot(cluster_me[y_kmeans==counter].mean())
    counter += 1
leftplot.set_xticks(range(0,4))
leftplot.set_xticklabels(["Family","Education","Second Half","Old Age"])
leftplot.set_xlabel("Migration Age Group")

rightplot.set_title("Number of Districts with Profile")
for idx in range(0,n):
    rightplot.bar(x      = idx,
                  height = (y_kmeans==idx).sum())
rightplot.set_xticks(range(0,4))
rightplot.set_xticklabels(["Neutral","Magnet","Repulsing","Attracting"])

plt.savefig("./images/Clusters.png")
plt.close()
#plt.show()

<img src = "./images/Clusters.png" />

<i>It seems the 4 Clusters are mostly differentiable by thier attraction to the Educational Migration Group. I divided the districts therefore into those four: Repulsing, Neutral, Attracting and Magnets.</i>

### Export Data for Demonstrator

In [ ]:
# Collect Data
for_export                 = predictionS(10,bc_dis_pop,mm_dis_pop)

data_export                = pd.DataFrame([])
data_export["name"]        = data_dis.name
data_export["code"]        = data_dis.gkz
data_export["pred_2018"]   = for_export.loc[:,2018].astype(int)
data_export["pred_2022"]   = for_export.loc[:,2022].astype(int)
data_export["pred_2026"]   = for_export.loc[:,2026].astype(int)
data_export["mig_profile"] = y_kmeans

In [ ]:
# Export Data
data_export.to_csv(path_or_buf = "data_demo.csv",
                   index = False)

<a id='Conclusion'></a>
## Conclusion

<i>Some of the results are conclusive, while others seem outright unrealistic. In the end, more data is needed, data that truely holds the possibility to determine the attractiveness of a municipality or district.<br>
In general the approach is feasable, it needs to be altered. Right now it is a purely "Data-Science" approach: Population is an infinite resource. In a future iteration the total population needs to be predicted, followed by a division into mobile, and immobile inhabitants. Then, the mobile ones are distributed among the districts/ municiplaities, based on a predicted attractiveness-factor.<br>
In the end, the lines between Data-Science and Simulation need to be blurred.</i>

<a id='FutureWorks'></a>
## Future Works

- Implement Geospatial Visualization of Municipality Level<br>
- Implement Categorical Feature: Type of District/ Municipality<br>
- Mature Missing Value Handling<br>
- Counter the influence of the Census 2011<br>